## Digit Recognizer 

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,Conv2D,MaxPool2D,Flatten,Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

## Loading Data & Data Preparation

In [ ]:
train = pd.read_csv("../input/digit-recognizer/train.csv")
test = pd.read_csv("../input/digit-recognizer/test.csv")

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
y_train = train["label"]
x_train = train.drop(labels=["label"],axis=1)

In [ ]:
x_train.head()

In [ ]:
y_train[0:5]

In [ ]:
y_train.value_counts()

In [ ]:
sns.countplot(x = y_train, palette  = "dark")

In [ ]:
img = x_train.iloc[0].values
img = img.reshape(28,28)
plt.imshow(img, cmap="brg")
plt.axis("off")
plt.title(train.iloc[0,0])
plt.show()

In [ ]:
img = x_train.iloc[100].values
img = img.reshape(28,28)
plt.imshow(img,cmap="gist_gray_r")
plt.title(train.iloc[100,0])
plt.axis("off")
plt.show()

In [ ]:
#In this study, the images will be used as gray scaled.

img = x_train.iloc[900].values
img = img.reshape(28,28)
plt.imshow(img,cmap="gray")
plt.title(train.iloc[900,0])
plt.axis("off")
plt.show()

### Normalization

In [ ]:
x_train = x_train / 255.0
test = test / 255.0

### Reshape

In [ ]:
x_train = x_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)
print("x_train shape: ", x_train.shape)
print("test shape: ", test.shape)

### Label Encoding

In [ ]:
y_train = to_categorical(y_train,num_classes = 10)

### Train-Test Split

In [ ]:
x_train,x_val,y_train,y_val = train_test_split(x_train,y_train,test_size = 0.1,random_state = 2)
print("x_train shape :",x_train.shape)
print("x_val shape :",x_val.shape)
print("y_train shape :",y_train.shape)
print("y_val shape :",y_val.shape)

## Convolutional Neural Network 

In [ ]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5), activation = "relu", padding = "Same", input_shape = (28,28,1)))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(filters = 16, kernel_size = (5,5), activation = "relu", padding = "Same"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256,activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10,activation = "softmax"))



In [ ]:
model.compile(optimizer = "RMSprop",
              loss = "categorical_crossentropy", 
              metrics = ["accuracy"])

In [ ]:
epochs = 10
batch_size = 250

### Data Augmentation

In [ ]:
datagen=ImageDataGenerator(
 featurewise_center=False,
 samplewise_center=False,
 featurewise_std_normalization=False,
 samplewise_std_normalization=False,
 zca_whitening=False,
 rotation_range=0.5,
 zoom_range=0.5,
 width_shift_range=0.5,
 height_shift_range=0.5,
 horizontal_flip=False,
 vertical_flip=False)


datagen.fit(x_train)

In [ ]:
model.summary()

## Model Fitting

In [ ]:
hist = model.fit(datagen.flow(x_train, y_train, batch_size = batch_size),
                      epochs = epochs,
                      validation_data = (x_val, y_val),
                      steps_per_epoch = x_train.shape[0] // batch_size)

## Evaluate the Model

In [ ]:

fig, ax = plt.subplots(2,1)
ax[0].plot(hist.history['loss'], color='b', label="Training loss")
ax[0].plot(hist.history['val_loss'], color='r', label="Validation loss",axes =ax[0])
legend = ax[0].legend()


ax[1].plot(hist.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(hist.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend()

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
y_predict=model.predict(x_val)
y_predict_classes=np.argmax(y_predict,axis=1)
y_true=np.argmax(y_val,axis=1)
confusion_mtx=confusion_matrix(y_true,y_predict_classes)
f,ax=plt.subplots(figsize=(8,8))
sns.heatmap(confusion_mtx,annot=True,linewidths=0.01,cmap="Purples",linecolor="gray",fmt=".1f",ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix")
plt.show()